In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [3]:
url = "https://statsapi.mlb.com/api/v2/people"

In [3]:
# get MLB teams
teams = requests.get('https://statsapi.mlb.com/api/v1/teams', params={'division' : 'Major League Baseball'}).json()
team_df = {}

for team in teams['teams']:
    try:
        if team['league']['name'] in ['National League', 'American League']:
            team_df[team['name']] = team['id']
    except KeyError:
        pass

mlb_teams = pd.DataFrame(team_df, index=[0]).T.reset_index()
mlb_teams.rename(columns={0: 'team_id', "index" :'team'}, inplace=True)
mlb_teams

,team,team_id
0,Los Angeles Angels,108
1,Arizona Diamondbacks,109
2,Baltimore Orioles,110
3,Boston Red Sox,111
4,Chicago Cubs,112
5,Cincinnati Reds,113
6,Cleveland Guardians,114
7,Colorado Rockies,115
8,Detroit Tigers,116
9,Houston Astros,117


In [5]:
players = {}

for team in mlb_teams['team_id']:
    current_team = requests.get('https://statsapi.mlb.com/api/v1/teams/' + str(team) + '/roster', 
                    params={'rosterType': 'active', 'season' : '2022'}).json()
    for player in current_team['roster']:
        current_player = {}
        current_player['id'] = player['person']['id']
        current_player['position'] = player['position']['abbreviation']
        current_player['status'] = player['status']['code']
        players[player['person']['fullName']] = current_player
players = pd.DataFrame(players).T.reset_index().rename(columns={'index' : 'player'})

In [52]:
position_players = players[(players['position'] != 'P') & (players['status'] == 'A')]
pitchers = players[(players['position'] == 'P') & (players['status'] == 'A')]

pitchers

,player,id,position,status
3,Jaime Barria,642545,P,A
4,Archie Bradley,605151,P,A
5,Jesse Chavez,445926,P,A
7,Reid Detmers,672282,P,A
15,Jimmy Herget,623474,P,A
...,...,...,...,...
1476,Peter Strzelecki,657265,P,A
1477,Brent Suter,608718,P,A
1480,Justin Topa,623437,P,A
1482,Devin Williams,642207,P,A


In [47]:
def create_position_player_stats(playerIDs : list, season : int, startCount = 0, limit = 1000000, minors = False):
    excluded_ids = []
    game_dict = []
    count = 0
    url = "https://statsapi.mlb.com/api/v1/people/"
    for player in playerIDs[startCount:]:
        if minors:
            current_player = requests.get(f"https://statsapi.mlb.com/api/v1/people/{player}/stats?stats=gameLog&gameType=R&leagueListId=milb_all&group=hitting&hydrate=team(league)&language=en&season={season}",).json()
        else:
            current_player = requests.get(url + str(player) + '/stats?season=' + str(season) + '&group=hitting&stats=gameLog',).json()
        # print(current_player)
        try:
            for game in current_player['stats'][0]['splits']:
                game_dict.append({"name": game['player']['fullName'],
                                                    "gameDate": game['date'],
                                                    "gameID": game['game']['gamePk'],
                                                    "team": game['team']['name'],
                                                    "opponent": game['opponent']['name'],
                                                    "team": game['team']['name'],
                                                    "hits": game['stat']['hits'],
                                                    "atBats": game['stat']['atBats'],
                                                    "homeRuns": game['stat']['homeRuns'],
                                                    "runs": game['stat']['runs'],
                                                    "rbi": game['stat']['rbi'],
                                                    "stolenBases": game['stat']['stolenBases'],
                                                    "strikeOuts": game['stat']['strikeOuts'],
                                                    "walks": game['stat']['baseOnBalls'],
                                                    "avg": game['stat']['avg'],
                                                    "obp": game['stat']['obp'],
                                                    "slg": game['stat']['slg'],
                                                    "ops": game['stat']['ops'],})
            count += 1
        except IndexError or gaierror:
            excluded_ids.append(player)
        
        if count % 1000 == 0:
            print(count)
        if count == limit:
            break
    return pd.DataFrame(game_dict), excluded_ids

In [28]:

aaa = requests.get(f'https://statsapi.mlb.com/api/v1/sports/{11}/players',).json()

# kwong['stats'][0]['splits'][2]
aaa = pd.DataFrame(aaa['people'])
position_players_minors = aaa[aaa["fullName"].isin(list(position_players["player"]))]
pitchers_minors = aaa[aaa["fullName"].isin(list(pitchers["player"]))]
pitchers_minors

,id,fullName,link,firstName,lastName,primaryNumber,birthDate,currentAge,birthCity,birthCountry,...,fullFMLName,fullLFMName,strikeZoneTop,strikeZoneBottom,birthStateProvince,draftYear,nameMatrilineal,nameTitle,nameSuffix,lastPlayedDate
1,676265,Cory Abbott,/api/v1/people/676265,Cory,Abbott,NaN,1995-09-20,27,San Diego,USA,...,Cory James Abbott,"Abbott, Cory James",3.467,1.589,CA,2017.0,NaN,NaN,NaN,NaN
3,656061,Albert Abreu,/api/v1/people/656061,Albert,Abreu,22,1995-09-26,27,Guayubin,Dominican Republic,...,Albert Enmanuel Abreu,"Abreu, Albert Enmanuel",3.467,1.589,NaN,NaN,Dias,NaN,NaN,NaN
14,672851,Joan Adon,/api/v1/people/672851,Joan,Adon,52,1998-08-12,24,Santo Domingo,Dominican Republic,...,Joan Manuel Adon,"Adon, Joan Manuel",3.467,1.589,NaN,NaN,NaN,NaN,NaN,NaN
24,669920,Jason Alexander,/api/v1/people/669920,Jason,Alexander,51,1993-03-01,30,Windsor,USA,...,Jason Raymond Alexander,"Alexander, Jason Raymond",3.467,1.589,CA,NaN,NaN,NaN,NaN,NaN
25,518397,Scott Alexander,/api/v1/people/518397,Scott,Alexander,39,1989-07-10,33,Santa Rosa,USA,...,Scott Alain Alexander,"Alexander, Scott Alain",3.467,1.589,CA,2010.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2162,642232,Ryan Yarbrough,/api/v1/people/642232,Ryan,Yarbrough,NaN,1991-12-31,31,Austin,USA,...,Ryan Christian Yarbrough,"Yarbrough, Ryan Christian",3.575,1.681,TX,2014.0,NaN,NaN,NaN,NaN
2164,489446,Kirby Yates,/api/v1/people/489446,Kirby,Yates,46,1987-03-25,35,Lihue,USA,...,Kirby Kali Yates,"Yates, Kirby Kali",3.301,1.504,HI,2005.0,NaN,NaN,NaN,NaN
2167,622065,Alex Young,/api/v1/people/622065,Alex,Young,34,1993-09-09,29,Westlake,USA,...,Alexander Edward Young,"Young, Alexander Edward",3.490,1.601,OH,2015.0,NaN,NaN,NaN,NaN
2181,669145,Bruce Zimmermann,/api/v1/people/669145,Bruce,Zimmermann,19,1995-02-09,28,Baltimore,USA,...,Bruce Anthony Zimmermann,"Zimmermann, Bruce Anthony",3.411,1.565,MD,2017.0,NaN,NaN,NaN,NaN


In [34]:
# position_stats, excluded_ids= create_position_player_stats(position_players.id, 2019)
position_stats, excluded_ids= create_position_player_stats(position_players.id, 2016, minors=True)

position_stats

0


,name,gameDate,gameID,team,opponent,hits,atBats,homeRuns,runs,rbi,stolenBases,strikeOuts,walks,avg,obp,slg,ops
0,Matt Duffy,2016-07-30,467650,Sacramento River Cats,Salt Lake Bees,1,2,0,0,0,0,0,0,.500,.500,.500,1.000
1,Matt Duffy,2016-07-31,467651,Sacramento River Cats,Salt Lake Bees,1,4,0,2,1,0,1,1,.333,.429,.500,.929
2,Matt Duffy,2016-08-04,457967,Charlotte Stone Crabs,Palm Beach Cardinals,1,3,0,0,0,0,0,0,.333,.333,.333,.666
3,Matt Duffy,2016-08-05,458014,Charlotte Stone Crabs,Fort Myers Miracle,2,3,0,0,1,0,0,0,.500,.500,.667,1.167
4,Matt Duffy,2016-08-07,458147,Durham Bulls,Gwinnett Braves,0,4,0,0,0,0,1,0,.200,.273,.300,.573
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20522,Kolten Wong,2016-06-11,467124,Memphis Redbirds,Colorado Springs Sky Sox,1,2,0,1,0,0,1,2,.500,.600,1.000,1.600
20523,Kolten Wong,2016-06-12,467125,Memphis Redbirds,Colorado Springs Sky Sox,1,3,0,2,1,0,1,0,.467,.579,1.000,1.579
20524,Kolten Wong,2016-06-13,467198,Memphis Redbirds,Nashville Sounds,1,4,0,0,0,0,1,1,.421,.542,.842,1.384
20525,Kolten Wong,2016-06-14,467199,Memphis Redbirds,Nashville Sounds,2,5,2,2,4,0,0,0,.417,.517,1.000,1.517


In [32]:
position_stats.groupby("name").size().sort_values()

name
Tyler Nevin           1
Charlie Blackmon      2
Aledmys Diaz          2
Avisail Garcia        3
Yan Gomes             3
                   ... 
Ian Happ            134
Brian Anderson      135
Matt Chapman        135
Rhys Hoskins        135
Ozzie Albies        138
Length: 276, dtype: int64

In [61]:
# position_players_list = []
years = [2022]
for year in years:
    position_players_list.append(create_position_player_stats(position_players.id, year, minors = True)[0])
position_minors_stats = pd.concat(position_players_list)

In [62]:
position_minors_stats = pd.concat(position_players_list)
position_minors_stats

,name,gameDate,gameID,team,opponent,hits,atBats,homeRuns,runs,rbi,stolenBases,strikeOuts,walks,avg,obp,slg,ops
0,David Fletcher,2015-06-18,427733,Orem Owlz,Ogden Raptors,1,3,0,0,1,0,0,1,.333,.500,.333,.833
1,David Fletcher,2015-06-19,427734,Orem Owlz,Ogden Raptors,1,5,0,0,1,0,0,0,.250,.333,.250,.583
2,David Fletcher,2015-06-20,427771,Orem Owlz,Ogden Raptors,1,4,0,0,1,0,0,1,.250,.357,.250,.607
3,David Fletcher,2015-06-21,427772,Orem Owlz,Ogden Raptors,1,4,0,2,0,1,1,1,.250,.368,.250,.618
4,David Fletcher,2015-06-22,427543,Orem Owlz,Grand Junction Rockies,4,5,0,1,1,2,0,0,.381,.458,.429,.887
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13986,Luis Urias,2022-04-29,674928,Biloxi Shuckers,Pensacola Blue Wahoos,0,4,0,0,0,0,3,0,.158,.273,.211,.484
13987,Luis Urias,2022-04-30,674927,Biloxi Shuckers,Pensacola Blue Wahoos,1,5,0,1,1,0,0,0,.167,.259,.208,.467
13988,Luis Urias,2022-05-01,674888,Biloxi Shuckers,Pensacola Blue Wahoos,0,3,0,0,0,0,0,0,.148,.258,.185,.443
13989,Kolten Wong,2022-06-25,671462,Wisconsin Timber Rattlers,Quad Cities River Bandits,0,4,0,1,0,0,0,0,.000,.000,.000,.000


In [ ]:
war

In [63]:
position_minors_stats.to_csv("2022_position_players.csv")